In [1]:
import pandas as pd
from ollama import chat
from ollama import ChatResponse
from tqdm import tqdm

In [2]:
df = pd.read_csv('Drug-ATC.csv')
SMILES = df['Neutralized SMILES']
ATC4 = df['ATC Codes']

In [3]:
s_smiles = SMILES[1].replace("\\", "\\\\") #L02AE
print(s_smiles)
response: ChatResponse = chat(model='atc4-predictor5', messages=[
    {"role": "user", "content": s_smiles}
])
print(response['message']['content'])

CC(C)C[C@H](NC(=O)[C@@H](COC(C)(C)C)NC(=O)[C@H](Cc1ccc(O)cc1)NC(=O)[C@H](CO)NC(=O)[C@H](Cc1c[nH]c2ccccc12)NC(=O)[C@H](Cc1cnc[nH]1)NC(=O)[C@@H]1CCC(=O)N1)C(=O)N[C@@H](CCCN=C(N)N)C(=O)N1CCC[C@H]1C(=O)NNC(N)=O
A01AA, A11BA, A11CA


In [4]:
import asyncio
from ollama import AsyncClient

async def chatA():
    message = {'role': 'user', 'content': SMILES[24].replace("\\", "\\\\")}
    response = await AsyncClient().chat(model='atc4-predictor5', messages=[message])
    print(response['message']['content'])

await chatA()

A11BA


In [16]:
if "predicted_atc4" not in df.columns:
    df["predicted_atc4"] = None
import asyncio
from ollama import AsyncClient

async def chat(message):
    response = await AsyncClient().chat(model='atc4-predictor5', messages=message)
    return response['message']['content']

for i in range(len(df)):
    smiles = df.at[i, "Neutralized SMILES"]
    name = df.at[i, "Molecule Names"]
    s_smiles = smiles.replace("\\", "\\\\")
    prompt = f"{name} - {s_smiles}"
    message=[
        {"role": "user", "content": prompt}
    ]
    try:
      pred = await chat(message)
    except ollama.ResponseError as e:
      print('Error:', e.error)
      if e.status_code == 404:
        ollama.pull(model)
    print(pred)
    df.at[i, "predicted_atc4"] = pred

    if i % 10 == 0:
        df.to_csv("predicted_atc4_partial.csv", index=False)
        print("Guardado parcial")

df.to_csv("predicted_atc4-NAME+SMILES.csv", index=False)

B01AA
Guardado parcial
H74BA
A01AA
A06AA, A11DA
D06AA, A11DA
A07BA, A11DA
A07BA, A07BB
H77BA, A11EA
A07BA
A08AA
A07BA
Guardado parcial
H04BD
H16AA, H02AA
A02AA, A02AB
A02AA
A02BA
A1A


A11CC, A11BA
A02AA, A02AB
A11AA, A11AC


NameError: name 'ollama' is not defined

In [5]:
response: ChatResponse = chat(model='atc4-predictor5', messages=[
    {'role': 'user', 'content': 'Cc1ncc(COP(=O)(O)O)c(C=O)c1O'}, # Pyridoxal phosphate - A11HA
    {'role': 'user', 'content': 'O=C(O)CCCC[C@@H]1CCSS1'} # Lipoic acid - A16AX
])

print(response['message']['content'])

A11AA, A11BA


In [6]:
response: ChatResponse = chat(model='atc4-predictor5', messages=[
    {'role': 'user', 'content': 'NCCCC[C@H](N)C(=O)O'},
])  # Lysine - B05XB

print(response['message']['content'])

A07AA


In [7]:
response: ChatResponse = chat(model='atc4-predictor5', messages=[
    {'role': 'user', 'content': 'C/C1=C2/N=C(/C=C3\\N=C(/C(C)=C4/[C@@H](CCC(N)=O)[C@](C)(CC(N)=O)[C@](C)([C@@H]5N=C1[C@](C)(CCC(=O)NC[C@@H](C)OP(=O)([O-])O[C@H]1[C@@H](O)[C@@H](n6cnc7cc(C)c(C)cc76)O[C@@H]1CO)[C@H]5CC(N)=O)N4[Co+]C#N)[C@@](C)(CC(N)=O)[C@@H]3CCC(N)=O)C(C)(C)[C@@H]2CCC(N)=O'},
]) # Vitamin B12 - B03AE; B03BA

print(response['message']['content'])

A01EA, A01EX
